In [ ]:
%matplotlib inline

# Climatology

As a first step in coordinate development, we'll work on sections of climatology from [WOA13](https://www.nodc.noaa.gov/OC5/woa13/). Because temperature is given in-situ, we first have to convert to potential temperature. Similar to [convert_WOA13](https://github.com/adcroft/convert_WOA13), we use the Python `gsw` package, which implements TEOS-10. We need to first convert practical salinity into absolute salinity, which we can then use in conjunction with in-situ temperature to calculate potential temperature. This ensures that we don't have spurious instabilities in water columns.

In [ ]:
from netCDF4 import Dataset
import matplotlib.pyplot as plt
import numpy as np
from remapping import mom_remapping
import gsw

As an initial set of input files, we load up THREDDS URLs for monthly averaged climatologies from 2005-2012.

In [ ]:
url_format = 'https://data.nodc.noaa.gov/thredds/dodsC/woa/WOA13/DATAv2/temperature/netcdf/A5B2/1.00/woa13_A5B2_{}{:02}_01v2.nc'

In [ ]:
temp_urls = [url_format.format('t', i+1) for i in range(12)]
salt_urls = [url_format.format('s', i+1) for i in range(12)]

Now we define a function that gives us a potential density section from a given month (default January), and range of latitudes and longitudes.

In [ ]:
def sect(lat, lon, month=0):
    salt_data = Dataset(salt_urls[month])
    temp_data = Dataset(temp_urls[month])
    
    depth = salt_data.variables['depth'][:]
    
    # compute absolute salinity
    sa = gsw.SA_from_SP(salt_data.variables['s_an'][0,:,lat,lon], depth, lon, lat)
    # compute conservative temperature
    ct = gsw.CT_from_t(sa, temp_data.variables['t_an'][0,:,lat,lon], depth, lon, lat)
    
    # compute potential density
    return gsw.rho(sa, ct, 0)

# Sections

Although our choice of coordinate should apply globally, we're particularly interested in a few troublesome spots, where there tend to always be problems, such as the Denmark Strait and the Sulu Sea. We may also care about dense overflows off Antarctica.